In [1]:
%reload_ext Cython

In [28]:
%%cython -a

import cStringIO
import numpy as np
from PIL import Image

IMAGE_SIZE = (128, 128)
IMAGE_ARRAY_SIZE = IMAGE_SIZE[0] * IMAGE_SIZE[1]

cdef c_ela_from_image(str file_path, int quality=95):
#     source_image
#     string_io
#     output_image_io
#     output_image
    
    source_image = Image.open(file_path)
    string_io = cStringIO.StringIO()
    source_image = source_image.resize(IMAGE_SIZE)
    source_image.save(string_io, 'JPEG', quality=quality)
    output_image_io = Image.open(string_io)
    output_image = Image.new('RGB', source_image.size)
    output_image.putdata(map(c_generator.calculate,
                             source_image.getdata(),
                             output_image_io.getdata()))
    output_image = output_image.convert('L')
    return np.r_[output_image].reshape(IMAGE_ARRAY_SIZE)

cdef class C_ELAGenerator:
    cdef int trigger, enhance
    cdef bint coloronly
    
    def __init__(self, int trigger=10, int enhance=20, bint coloronly=False):
        self.trigger = trigger
        self.enhance = enhance
        self.coloronly = coloronly

    def calculate(self, pixelA, pixelB):
        pixelDiff = map(lambda x, y: abs(x - y), pixelA, pixelB)
        if sum(pixelDiff) > self.trigger and (not self.coloronly or pixelDiff[0] != pixelDiff[1] or pixelDiff[0] != pixelDiff[2]):
            return tuple([x * self.enhance for x in pixelDiff])
        else:
            return (0, 0, 0)



c_generator = C_ELAGenerator(trigger=1, enhance=50)

In [3]:
import cStringIO
import numpy as np
from PIL import Image

IMAGE_SIZE = (128, 128)
IMAGE_ARRAY_SIZE = IMAGE_SIZE[0] * IMAGE_SIZE[1]

def ela_from_image(file_path, quality=95):
    source_image = Image.open(file_path)
    string_io = cStringIO.StringIO()
    source_image = source_image.resize(IMAGE_SIZE)
    source_image.save(string_io, 'JPEG', quality=quality)
    output_image_io = Image.open(string_io)
    output_image = Image.new('RGB', source_image.size)
    output_image.putdata(map(generator.calculate,
                             source_image.getdata(),
                             output_image_io.getdata()))
    output_image = output_image.convert('L')
    return np.r_[output_image].reshape(-1)

class ELAGenerator():
    def __init__(self, trigger=10, enhance=20, coloronly=False):
        self.trigger = trigger
        self.enhance = enhance
        self.coloronly = coloronly

    def calculate(self, pixelA, pixelB):
        pixelDiff = map(lambda x, y: abs(x - y), pixelA, pixelB)
        if sum(pixelDiff) > self.trigger and \
            (not self.coloronly or \
             pixelDiff[0] != pixelDiff[1] or pixelDiff[0] != pixelDiff[2]):
            return tuple([x * self.enhance for x in pixelDiff])
        else:
            return (0, 0, 0)



generator = ELAGenerator(trigger=1, enhance=50)

In [4]:
import os
from sklearn import cross_validation

def image_files_in_folder(folder):
    all_files = os.listdir(folder)
    path_list = []
    for file in all_files:
        if file.endswith('.jpg'):
            path_list.append('%s/%s' % (folder, file))
    return path_list


np.random.seed(0)
# psed_file_names = image_files_in_folder('psed-by-email')[:500]
# nonpsed_file_names = image_files_in_folder('non-psed')[:500]
# X = np.concatenate((psed_file_names,
#                     nonpsed_file_names))
# y = np.concatenate((np.repeat(1, len(psed_file_names)),
#                     np.repeat(0, len(nonpsed_file_names))))

# X_train, X_test, y_train, y_test = \
#     cross_validation.train_test_split(X, y, random_state=0, train_size=.8)


In [11]:
path = image_files_in_folder('../data/non-psed')[0]

%timeit ela_from_image(path)
%timeit c_ela_from_image(path)

1 loop, best of 3: 362 ms per loop
1 loop, best of 3: 339 ms per loop
